## Training für das Cut in Szenario

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
import environment.test_env as Env 

In [ ]:
import agent.ddpg_tf2 as ddpg_tf2

In [ ]:
agent = ddpg_tf2.Agent((2,))

In [ ]:
rewards_list = []
returns_list = []
steps_list = []
init_values = []
modes = []
velocities = []

In [ ]:
best_score = 150
score_history = []
old_score = 0
all_steps = 0
i = 1

while all_steps < 500000:
    initial_speed_ego = np.random.uniform(low = 18.0, high = 30.0) 
    initial_speed_lead = np.random.uniform(low = 20.0, high = 28.0)
    inital_speed_goal = np.random.uniform(low = 20.0, high = 28.0)
    initial_gap = np.random.uniform(low = 25.0, high = 60.0)
    initial_safety_gap = initial_speed_lead * 3.6 / 2
    
    init_values.append([initial_gap, initial_safety_gap, initial_speed_ego, initial_speed_lead])

    env = Env.VehicleEnvironment(initial_speed_ego, initial_speed_lead, initial_gap, initial_safety_gap)
    observation = np.array(env.get_state())

    done = False
    score = 0
    steps = 0
    vs = []
    
    for j in range(200):
        vs.append(env.ego_velocity)
        all_steps += 1
        steps += 1
        action = agent.choose_action(observation)
        action_ = action.numpy()[0]
        observation_, reward, done = env.step(action_, j)
        score += reward
        observation_ = np.array([observation_])
        reward = np.array([[reward]])
        done = np.array([[done]])
        agent.remember(observation, action, reward, observation_, done)
        agent.learn()
        observation = np.array(observation_[0])
        rewards_list.append(reward)
        
        if done:
            break
    
    # save data 
    returns_list.append(score)
    steps_list.append(steps)
    modes.append(env.mode)
    velocities.append(vs)
    
    score_history.append(score)
    avg_score = np.mean(score_history[-100:])

    if score > best_score:
        best_score = score # score statt avg_score
        agent.save_models()

    print('episode ', i, 'score %.1f' % score, 'avg score %.1f' % avg_score)
    i += 1
    print('steps: ', steps)
    print(all_steps/500000, " %")

## Ergebnisse

In [ ]:
import utils

In [ ]:
velocities = []
accels = []
gaps = []
velocities_lead = []
safety_gaps = []
initial_speed_ego = np.random.uniform(low = 18.0, high = 30.0) 
initial_speed_lead = np.random.uniform(low = 20.0, high = 28.0)  
initial_gap = np.random.uniform(low = 25.0, high = 60.0)
initial_safety_gap = initial_speed_lead * 3.6 / 2

print("-----Initial Values-----")
print("gap: ", initial_gap)
print("safety gap: ", initial_safety_gap)
print("speed ego: ", initial_speed_ego)
print("speed lead: ", initial_speed_lead)

env = Env.VehicleEnvironment(initial_speed_ego, initial_speed_lead, initial_gap, initial_safety_gap)
print("-----Mode------")
print(env.mode)
# env.reset()
observation = np.array(env.get_state())
done = False
score = 0
for j in range(200):
        #print(observation)
        action = agent.choose_action(observation, True)
        action_ = action.numpy()[0]
        observation_, reward, done = env.step(action_, j)
        #print(observation_, reward, done)
        #print(action_)
        #print(env.get_state())
        velocities.append(env.ego_velocity)
        accels.append(action_)
        gaps.append(env.gap)
        safety_gaps.append(env.safety_gap)
        score += reward
        observation = np.array(observation_)
        #print(observation)
        velocities_lead.append(env.lead_velocity)
        if done:
            break
print("-----score-----")
print(score)

In [ ]:
# Ego Geschwindigkeit
utils.plot_speed(velocities, "graphs")

In [ ]:
# Lead Geschwindigkeit
utils.plot_speed(velocities_lead, "graphs")

In [ ]:
# Ego Beschleunigung
utils.plot_acceleration(accels, "graphs")

In [ ]:
# Abweichung zum Optimalabstand
g = []
for i in range(200):
    g.append(gaps[i] - safety_gaps[i])
utils.plot_gap(g, "graphs")

In [ ]:
# Returns
utils.plot_return(returns_list, "graphs")

## Daten speichern

In [ ]:
import pickle

In [ ]:
with open('steps.pkl', 'wb') as file:
    pickle.dump(steps_list, file)
    
with open('rewards.pkl', 'wb') as file:
    pickle.dump(rewards_list, file)

with open('returns.pkl', 'wb') as file:
    pickle.dump(returns_list, file)
    
with open('buffer.pkl', 'wb') as file:
    pickle.dump(agent.memory, file)
    
with open('init.pkl', 'wb') as file:
    pickle.dump(init_values, file)
    
with open('modes.pkl', 'wb') as file:
    pickle.dump(modes, file)
    
with open('velocities.pkl', 'wb') as file:
    pickle.dump(velocities, file)

In [ ]:
agent.target_actor.save_weights("last_taractor.h5")
agent.actor.save_weights("last_actor.h5")
agent.critic.save_weights("last_critic.h5")
agent.target_critic.save_weights("last_tarcritic.h5")